# Setup

In [1]:
import torch
from torch import nn
import random
from torchvision import datasets
from torchvision.transforms import ToTensor
from tqdm import tqdm
import json
import scipy.io
from torchmetrics import Accuracy

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

In [2]:
from IPython.display import clear_output
import os, sys, shutil
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# from tqdm import tqdm, trange
from glob import glob
import random
import cv2 as cv
from audiomentations import *

In [3]:
print(os.getcwd())

/home/linhpika/git/HeartResearch/Experiment/Approach/Model_signal


In [4]:
for i in range(3):
    os.chdir("..")

print(os.getcwd())

/home/linhpika/git/HeartResearch


In [5]:

main_data_dir = os.getcwd() + "/Experiment"



# lead = ['I','II','III','aVR','aVL','aVF','V1','V2','V3','V4','V5','V6']
lead = [0,1,2,3,4,5,6,7,8,9,10,11]
lead_to_onehot = {leads : torch.nn.functional.one_hot(torch.tensor([index])[0], num_classes = 12) for index, leads in enumerate(lead)}

class_la = [1,2,3,4,5,6,7,8,9]
class_to_onehot = {classes : torch.nn.functional.one_hot(torch.tensor([index])[0], num_classes = 9) for index, classes in enumerate(class_la)}

dataset_dir = os.getcwd() + "/Data set"
data_mat_list = glob(dataset_dir + "/TrainingSet*/*")
label_csv_path = dataset_dir + "/Label.csv"
print(len(data_mat_list))
print(data_mat_list[0])
print(type(data_mat_list[0]))

6877
/home/linhpika/git/HeartResearch/Data set/TrainingSet3/A6594.mat
<class 'str'>


In [6]:
lead_to_onehot[1]

tensor([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [7]:
label_df = pd.read_csv(label_csv_path)
label_df.head()

,Recording,First_label,Second_label,Third_label
0,A0001,5,NaN,NaN
1,A0002,1,NaN,NaN
2,A0003,2,NaN,NaN
3,A0004,2,NaN,NaN
4,A0005,7,NaN,NaN


In [8]:
#

 # Data Loader

In [9]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.nn.functional import one_hot

In [10]:
torch.cuda.is_available()

True

In [11]:
ratio = [0.8, 0.1, 0.1]

# random.seed(42)
# img_data_list = random.sample(img_data_list, k = 55000)

train_index = int(len(data_mat_list)*ratio[0])
valid_index = int(len(data_mat_list)*(ratio[0]+ratio[1]))

train_signal_paths = data_mat_list[:train_index]
valid_signal_paths = data_mat_list[train_index:valid_index]
test_signal_paths = data_mat_list[valid_index:]

train_label = label_df.iloc[:train_index,:]
valid_label = label_df.iloc[train_index:valid_index,:]
test_label = label_df.iloc[valid_index:,:]

print(len(train_signal_paths))
print(len(valid_signal_paths))
print(len(test_signal_paths))

print(train_label.shape)
print(valid_label.shape)
print(test_label.shape)

5501
688
688
(5501, 4)
(688, 4)
(688, 4)


In [12]:
# train_image_paths[0]

In [13]:
train_label

,Recording,First_label,Second_label,Third_label
0,A0001,5,NaN,NaN
1,A0002,1,NaN,NaN
2,A0003,2,NaN,NaN
3,A0004,2,NaN,NaN
4,A0005,7,NaN,NaN
...,...,...,...,...
5496,A5497,8,NaN,NaN
5497,A5498,1,NaN,NaN
5498,A5499,1,NaN,NaN
5499,A5500,5,NaN,NaN


In [14]:
# normalize = torch.nn.Sequential(transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)))

class HeartData(Dataset):
    def __init__(self, label_df, data_path):
        self.label_df = label_df
        self.data_path = data_path
        
#         self.onehot_label_class = one_hot(self.label_df['Class'])
#         self.onehot_label_lead = one_hot(self.label_df['Lead'])
    def __len__(self):
        return len(self.data_path)
        
    def __getitem__(self, index):
        filename = self.data_path[index].split("/")[-1].split(".")[0]
        class_label = self.label_df[self.label_df["Recording"] == filename]["First_label"].values.item()
        
        augment = Compose([
            AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.2),
            AddGaussianSNR(min_snr_db=3.0, max_snr_db=15.0, p=0.2),
            Gain(min_gain_db = -15.0, max_gain_db = 15.0, p =0.2),
            GainTransition(min_gain_db = -15.0, max_gain_db = 15.0, 
                           min_duration = 50, max_duration = 200, duration_unit = "samples", 
                           p = 0.2),
            TimeStretch(min_rate=0.8, max_rate=1.25, p=0.2),
            PitchShift(min_semitones=-4, max_semitones=4, p=0.2),
            Shift(min_fraction=-0.5, max_fraction=0.5, p=0.2),
        ])
        
        tensor_lst = []
        for i in range(12):
#             samples = data_mat_list[index]
            samples = self.data_path[index]
          
            # Create a sample signal (1D tensor)
            mat_data = scipy.io.loadmat(samples)  # Replace 'signal_data.mat' with your file path
            signal_array = mat_data['ECG'][0][0][2][i]  # 'signal' is the variable name in the .mat file
            augmented_samples = augment(samples=signal_array, sample_rate=500)

            # Define the desired length after padding
            desired_length = 30000

            # Calculate the amount of zero-padding needed
            pad_length = max(desired_length - len(augmented_samples), 0)
            
            if pad_length == 0:
                padded_augment = augmented_samples[:desired_length]
            else:
                # Perform zero-padding at the end of the signal
                padded_augment = np.pad(augmented_samples, (0, pad_length), mode='constant')

            augment_signal_tensor = torch.tensor(padded_augment)
            tensor_lst.append(augment_signal_tensor)
            
        n = torch.stack(tensor_lst, 1)
        
#         imgs = []
#         labels = []
#         labels.append(self.onehot_label_class)
#         labels.append(self.onehot_label_lead)
#         class_label = class_to_onehot[class_label]
    
#         data_img = cv.imread(self.data_path[index])
#         torch_img = torch.from_numpy(data_img).permute(-1, 0, 1).float()
#         torch_img = normalize(torch_img)
        
#         lead_label = self.onehot_label_lead[index]
#         class_label = self.onehot_label_class[index]
        return class_label, n

In [15]:
# class_label = train_label['Class'][0]
# class_label = class_to_onehot[class_label]
# print(type(class_label))
# data_img = cv.imread(train_image_paths[1])
# torch_img = torch.from_numpy(data_img).permute(-1, 0, 1)
# print(type(torch_img))
# print(torch_img.shape)

In [29]:
a = [1,2,3,4,5,6]
b = np.pad(a, (0, 5), mode='constant')
print(b)

[1 2 3 4 5 6 0 0 0 0 0]


In [16]:
train_dataset = HeartData(label_df, train_signal_paths)
valid_dataset = HeartData(label_df, valid_signal_paths)
test_dataset = HeartData(label_df, test_signal_paths)

In [17]:
# print(train_dataset[0])
# print(train_dataset[0][1].shape)
# print(train_dataset[1])
# print(train_dataset[1][1].shape)
# print(train_dataset[32])
# print(train_dataset[32][1].shape)
# print(max(len(train_dataset[i][1]) for i in range (len(train_dataset))))
print(len(train_dataset[32][1]))

/home/linhpika/git/.env/lib/python3.10/site-packages/audiomentations/core/transforms_interface.py:61: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(


30000


/home/linhpika/git/.env/lib/python3.10/site-packages/audiomentations/core/transforms_interface.py:61: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(


In [18]:
len(valid_dataset)

688

In [19]:
# valid_dataset[5]

In [20]:
train_dataloader = DataLoader(train_dataset, batch_size = 128, shuffle = True, pin_memory = True, num_workers = 48)
valid_dataloader = DataLoader(valid_dataset, batch_size = 128, shuffle = True, pin_memory = True, num_workers = 48)
test_dataloader = DataLoader(test_dataset, batch_size = 128, shuffle = True, pin_memory = True, num_workers = 48)

In [21]:
len(train_dataloader)

43

In [22]:
# Display image and label.
train_features, train_labels = next(iter(train_dataloader))
print(f" {train_features.size()}")
print(f" {train_labels.size()}")

 torch.Size([128])
 torch.Size([128, 30000, 12])


In [23]:
# multi-task learning - efficient B0
# data loader -> 2 labels: lead + disease
# Model: 2 output: 1 vector for 12 leads (softmax) + 1 vector for disease
# loss funct: loss lead + loss class => backward
# random choice: notice: seed(python, numpy, torch) same
        
# multi-channel - efficient B2 - quite similar to video classification

# Model 

In [24]:
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torch import nn

ori_model = efficientnet_b0(weights = EfficientNet_B0_Weights.IMAGENET1K_V1)

In [25]:
# ori_model

In [26]:
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torch import nn

class HeartModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.ori_model = efficientnet_b0(weights = EfficientNet_B0_Weights.IMAGENET1K_V1)
        del self.ori_model.classifier
        self.ori_model.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(1280, 9),
            nn.Softmax(dim = 1)
        )
        
    def forward(self, x):        
        logits = self.ori_model(x)
        return (logits)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu", index = 0)
model = HeartModel().to(device)
print(model)

HeartModel(
  (ori_model): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              (scale_

# Training

In [27]:
from datetime import datetime
from torch import optim
from torch.optim import lr_scheduler
opt_mapping = {
    "Adam" : torch.optim.Adam
}

loss_mapping = {
    "CCE" : nn.CrossEntropyLoss()
}

class Training:
    def __init__(self, 
                 device: str = "cpu",
                 learning_rate:float = 0.0001,
                 optimizer:str = "Adam",
                 loss:str = "CCE",
                 model = model,
#                  batchsize:int = 32,
                 epochs:int = 100,
#                  label_df: pd.DataFrame = label_df, 
#                  root_dir: str = img_data_dir, 
#                  ratio: tuple = (0.8, 0.1, 0.1),
#                  resize: tuple = None, #  (256, 512)
#                  seed: int = 777
                ):
        
        # Setup
        self.device = device
        self.model = model
        self.model.to(self.device)
        self.lr = learning_rate
        self.ep = epochs
        self.optimizer = opt_mapping[optimizer](self.model.parameters(), lr=self.lr)
        self.scheduler = lr_scheduler.CosineAnnealingLR(optimizer=self.optimizer, T_max=self.ep)
        self.loss_fn = loss_mapping[loss]
#         self.bs = batchsize
#         self.default_ratio = (0.001, 0.001, 0.001)
        self.cls_accuracy = Accuracy(task="multiclass", num_classes=9).to(self.device)
#         self.target_names = [str(i) for i in range(9)]
        
        # Data
        self.train_data = train_dataloader
        self.valid_data = valid_dataloader
        self.test_data = test_dataloader

    
    def __update__(self):
        train_losses = []
        train_cls_acc_plot = []
        
        val_losses = []
        val_cls_acc_plot = []
        
        for e in range(self.ep):
            correct = 0
            batch_cnt = 0
            self.model.train()
            for batch, (y, X) in tqdm(enumerate(self.train_data)):
                batch_cnt = batch
                y = y.to(self.device)
                pred = self.model(X.to(self.device))
                train_loss = self.loss_fn(pred, y)
                
                self.optimizer.zero_grad()
                train_loss.backward()
                self.optimizer.step()
                self.scheduler.step()
                
                correct += (pred.argmax(1) == y).type(torch.float).sum().item()
                        
            mean_train_cls_acc = correct/len(self.train_data.dataset)*100
  
            train_losses.append(train_loss)
            train_cls_acc_plot.append(mean_train_cls_acc)

            print(f"Epoch: {e} - Train Loss: {train_loss.item()} - Train class acc: {mean_train_cls_acc}")
            
            val_loss, mean_valid_cls_acc = self.validation(epoch = e)
            val_losses.append(val_loss)
            val_cls_acc_plot.append(mean_valid_cls_acc)
            # Show train_loss, train_acc, val_loss, val_acc
            if e%9 == 0: 
                self.loss_plot(train_losses = train_losses, 
                          val_losses = val_losses, n_epochs = e + 1, 
                          check_folder = 'run_efficientB0',
                              task_folder = 'Singletask')
                self.cls_acc_plot(train_cls_acc = train_cls_acc_plot , 
                                  val_cls_acc = val_cls_acc_plot, 
                                  n_epochs = e+1, 
                                  check_folder = 'run_efficientB0',
                                 task_folder = 'Singletask')
               
        self.evaluation(check_folder = 'run_efficientB0', task_folder = 'Singletask')
        
    def validation(self, epoch):
        
        self.model.eval()
        correct = 0
        batch_cnt = 0
        
        with torch.no_grad():
            valid_class_acc = 0
            valid_lead_acc = 0
            batch_cnt = 0
            for batch, (y, X) in tqdm(enumerate(self.valid_data)):
                batch_cnt = batch

                y = y.to(self.device)
                pred = self.model(X.to(self.device))

                val_loss = self.loss_fn(pred, y)

                correct += (pred.argmax(1) == y).type(torch.float).sum().item()                

            mean_valid_cls_acc = correct/(len(self.valid_data.dataset))*100
            
            self.checkpoint(valid_class_acc = mean_valid_cls_acc, 
                       val_total_loss = val_loss,
                       epoch = self.ep, 
                       model = self.model,
                       optimizer = self.optimizer,
                       check_folder = 'run_efficientB0',
                           task_folder = 'Singletask')

            print(f"Epoch: {epoch} - Val_loss: {val_loss.item()} - Val class acc: {mean_valid_cls_acc}")
        
        return val_loss, mean_valid_cls_acc

    def loss_plot(self, train_losses, val_losses, n_epochs, check_folder, task_folder):
        now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
        run_dir = os.getcwd() + f"/{check_folder}"
        if not os.path.exists(run_dir):
            os.mkdir(run_dir)
            
        task_dir = run_dir + f"/{task_folder}"
        if not os.path.exists(task_dir):
            os.mkdir(task_dir)
            
        save_loss_dir = task_dir + "/save_losses"
        if not os.path.exists(save_loss_dir):
            os.mkdir(save_loss_dir)
        save_fig_losses = os.path.join(save_loss_dir, f"plot_losses_epoch{n_epochs}_{now}.png")  
        
        plt.figure()
        plt.plot(range(1, n_epochs + 1), torch.tensor(train_losses).cpu(), label='Train Loss')
        plt.plot(range(1, n_epochs + 1), torch.tensor(val_losses).cpu(), label='Validation Loss')
        plt.xlabel('Epoch')
        # plt.xticks()
        plt.ylabel('Loss')
        plt.title('Training and Validation Loss')
        plt.legend()
        plt.savefig(save_fig_losses)
        
    def cls_acc_plot(self, train_cls_acc, val_cls_acc, n_epochs, check_folder, task_folder):
        now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
        run_dir = os.getcwd() + f"/{check_folder}"
        if not os.path.exists(run_dir):
            os.mkdir(run_dir)
            
        task_dir = run_dir + f"/{task_folder}"
        if not os.path.exists(task_dir):
            os.mkdir(task_dir)
            
        save_acc_dir = task_dir + "/save_acc"
        if not os.path.exists(save_acc_dir):
            os.mkdir(save_acc_dir)
        save_fig_acc = os.path.join(save_acc_dir, 'plot_cls_acc_epoch{}_{}.png'.format(n_epochs, now))  
        
        plt.figure()
        plt.plot(range(1, n_epochs + 1), torch.tensor(train_cls_acc).cpu(), label='Train Accuracy')
        plt.plot(range(1, n_epochs + 1), torch.tensor(val_cls_acc).cpu(), label='Validation Accuracy')
        plt.xlabel('Epoch')
        # plt.xticks
        plt.ylabel('Accuracy')
        plt.title('Training and Validation Accuracy')
        plt.legend()
        plt.savefig(save_fig_acc)

    
    def checkpoint(self, valid_class_acc, 
                   val_total_loss,
                   epoch, 
                   model,
                   optimizer,
                   check_folder,
                   task_folder
                  ):
        old_valid_class_acc = 0
        old_valid_loss = 1e23
        if valid_class_acc >= old_valid_class_acc and val_total_loss <= old_valid_loss:
            old_valid_class_acc = valid_class_acc
            old_valid_loss = val_total_loss
            save_dict = {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': val_total_loss,
                'val_class_acc': valid_class_acc
            }
            
         # Saving best model
            now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
            run_dir = os.getcwd() + f"/{check_folder}"
            if not os.path.exists(run_dir):
                os.mkdir(run_dir)
            task_dir = run_dir + f"/{task_folder}"
            if not os.path.exists(task_dir):
                os.mkdir(task_dir)
            save_best_model_dir = task_dir + "/save_best_model"
            if not os.path.exists(save_best_model_dir):
                os.mkdir(save_best_model_dir)
            save_best_model_path = save_best_model_dir + f"/{save_dict['loss']:>7f}_{save_dict['val_class_acc']:>7f}_{now}.pt"
            torch.save(save_dict, save_best_model_path)
    
    def evaluation(self, check_folder, task_folder):
        
        self.model.eval()
        
        y_eval = [] 
        pred_eval = []
        
        with torch.no_grad():
            for batch, (y, X) in tqdm(enumerate(self.test_data)):
                batch_cnt = batch
                
                y = y.to(self.device)
                pred = self.model((X/255).to(self.device))
                
                y_eval.append(torch.argmax(y, dim=1)[0])
                pred_eval.append(torch.argmax(pred, dim=1)[0])
                
            y_eval = torch.stack(y_eval)
            pred_eval = torch.stack(pred_eval)
                
            pred = torch.nn.functional.one_hot(pred_eval, num_classes = 9)
            y = torch.nn.functional.one_hot(y1_eval, num_classes = 9)

            
            # Classification report
            for i in range (len(class_la)):
                class_la[i] = str(class_la[i])
                
            labels = class_la
            now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
            run_dir = os.getcwd() + f"/{check_folder}"
            if not os.path.exists(run_dir):
                os.mkdir(run_dir)
                
            task_dir = run_dir + f"/{task_folder}"
            if not os.path.exists(task_dir):
                os.mkdir(task_dir)

            save_report_dir = task_dir + "/save_classification_report"
            if not os.path.exists(save_report_dir):
                os.mkdir(save_report_dir)

            ground_truth_class = torch.argmax(y, dim=1).cpu().numpy().tolist()
            predictions_class = torch.argmax(pred, dim=1).cpu().numpy().tolist()
            
            precision_class, recall_class, f_score_class, support_class = precision_recall_fscore_support(ground_truth_class,
                                                                          predictions_class,
                                                                          labels=labels,
                                                                          average=None)
            class_results_pd = pd.DataFrame({"class": labels,
                               "precision": precision_class,
                               "recall": recall_class,
                               "f_score": f_score_class,
                               "support": support_class
                               })
            save_class_report_path = os.path.join(save_report_dir, 'class_classification_report_{}.png'.format(now))
            class_results_pd.to_csv(save_class_report_path, index=False)
    

            # Confusion Matrix
            now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
            run_dir = os.getcwd() + f"/{check_folder}"
            if not os.path.exists(run_dir):
                os.mkdir(run_dir)
                
            task_dir = run_dir + f"/{task_folder}"
            if not os.path.exists(task_dir):
                os.mkdir(task_dir)

            save_CM_dir = task_dir + "/save_CM"
            if not os.path.exists(save_CM_dir):
                os.mkdir(save_CM_dir)
                
            save_cls_cm_path = os.path.join(save_CM_dir, 'class_CM_{}.png'.format(now))
            
            ConfusionMatrixDisplay.from_predictions(torch.argmax(y, dim=1).cpu().numpy().tolist(), torch.argmax(pred, dim=1).cpu().numpy().tolist(), cmap = "PuBuGn")
            plt.savefig(save_cls_cm_path)


            # ROC Curve

            for i in range(9):
                r2 = roc_auc_score(y[:, i].cpu(), pred[:, i].cpu())
                print("The ROC AUC score of "+ class_la[i] +" is: "+str(r2))


            # Compute ROC curve and ROC area for each class
            class_fpr = {}
            class_tpr = {}
            class_roc_auc = dict()
            
            now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
            run_dir = os.getcwd() + f"/{check_folder}"
            if not os.path.exists(run_dir):
                os.mkdir(run_dir)
                
            task_dir = run_dir + f"/{task_folder}"
            if not os.path.exists(task_dir):
                os.mkdir(task_dir)

            save_ROC_dir = task_dir + "/save_ROC"
            if not os.path.exists(save_ROC_dir):
                os.mkdir(save_ROC_dir)
                
            save_cls_ROC_path = os.path.join(save_ROC_dir, 'cls_ROC_{}.png'.format(now))
            
            for i in range(9):
                class_fpr[i], class_tpr[i], _ = roc_curve(y[:, i].cpu(), pred[:, i].cpu(), drop_intermediate=False)
                roc_auc[i] = auc(class_fpr[i], class_tpr[i])

            plt.plot(class_fpr[0], class_tpr[0],'turquoise',label='1: ROC curve of class 1 (area = %0.2f)' % roc_auc[0])
            plt.plot(class_fpr[1], class_tpr[1],'peachpuff',label='2: ROC curve of class 2 (area = %0.2f)' % roc_auc[1])
            plt.plot(class_fpr[2], class_tpr[2],'paleturquoise',label='3: ROC curve of class 3 (area = %0.2f)' % roc_auc[2])
            plt.plot(class_fpr[3], class_tpr[3],'pink',label='4: ROC curve of class 4 (area = %0.2f)' % roc_auc[3])
            plt.plot(class_fpr[4], class_tpr[4],'lightcoral',label='5: ROC curve of class 5 (area = %0.2f)' % roc_auc[4])
            plt.plot(class_fpr[5], class_tpr[5],'peachpuff',label='6: ROC curve of class 6 (area = %0.2f)' % roc_auc[5])
            plt.plot(class_fpr[6], class_tpr[6],'steelblue',label='7: ROC curve of class 7 (area = %0.2f)' % roc_auc[6])
            plt.plot(class_fpr[7], class_tpr[7],'forestgreen',label='8: ROC curve of class 8 (area = %0.2f)' % roc_auc[6])
            plt.plot(class_fpr[8], class_tpr[8],'darkslategray',label='9: ROC curve of class 9 (area = %0.2f)' % roc_auc[6])

            plt.plot([0, 1], [0, 1], 'k--')
            plt.xlim([-0.1, 1.1])
            plt.ylim([-0.1, 1.1])
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.title('Receiver operating characteristic of lead')
            plt.legend(loc="lower right")
            plt.savefig(save_cls_ROC_path)         

In [28]:
# Metadata

# Training Setup
device = "cuda"
learning_rate = 0.001
optimizer = "Adam"
loss = "CCE"
# batchsize = 128
epochs = 100
# ratio = (0.8, 0.1, 0.1)

# Data

monitor = Training(device = device,
                   learning_rate = learning_rate,
                   optimizer = optimizer,
                   loss = loss,
                   model = model,
#                    batchsize = batchsize,
                   epochs = epochs)
#                    label_df = label_df, 
#                    root_dir = img_data_dir, 
#                    ratio = ratio)
monitor.__update__()

0it [00:10, ?it/s]


RuntimeError: Given groups=1, weight of size [32, 3, 3, 3], expected input[1, 128, 30000, 12] to have 3 channels, but got 128 channels instead

In [ ]:
# Unitest

# monitor.get_sample_count()

In [ ]:
# monitor.update()

# Evaluation

In [ ]:
# monitor.evaluation()

# Clear Buffer

# monitor.clear_buffer()